In [4]:
import pennylane as qml
dev = qml.device('default.qubit',wires=1)

In [4]:
import pennylane as qml

H = qml.Hamiltonian([1, 1, 0.5], [qml.PauliX(0), qml.PauliZ(1), qml.PauliX(0) @ qml.PauliX(1)]) #定义了一个哈密顿量
print(H)


1 * X(0) + 1 * Z(1) + 0.5 * (X(0) @ X(1))


这段代码定义了一个哈密顿量 $\mathbf{H}$：

$$
H = 1 \cdot X_0 + 1 \cdot Z_1 + 0.5 \cdot (X_0 X_1)
$$

---

- `qml.Hamiltonian([1, 1, 0.5], [...])}`：定义了哈密顿量的系数。

  - $X_0$：第 0 个量子比特上的 Pauli-X 操作。
  - $Z_1$：第 1 个量子比特上的 Pauli-Z 操作。
  - $X_0 X_1$：第 0 和 1 号量子比特上的 Pauli-X 交互（XX 作用）。


In [3]:
import pennylane as qml
dev = qml.device("default.qubit", wires=2)

t = 1
n = 2


@qml.qnode(dev)
def circuit():
    qml.ApproxTimeEvolution(H, t, n)
    return [qml.expval(qml.PauliZ(i)) for i in range(2)]


print(qml.draw(circuit, level="device")())

0: ──RX(1.00)─╭RXX(0.50)──RX(1.00)─╭RXX(0.50)─┤  <Z>
1: ──RZ(1.00)─╰RXX(0.50)──RZ(1.00)─╰RXX(0.50)─┤  <Z>


这段代码使用 **PennyLane** 进行 **量子哈密顿量 \( H \) 的近似时间演化**，并输出相应的 **量子电路结构**。



$$
H = 1 \cdot X_0 + 1 \cdot Z_1 + 0.5 \cdot (X_0 X_1)
$$

---

 **代码解析**
```python
dev = qml.device("default.qubit", wires=2)
```

创建一个 **2 量子比特的 PennyLane 设备**，用于 **模拟量子计算机**。


`t = 1`：设定 **时间演化时间** \( t = 1 \)。

`n = 2`：设定 **Trotter 分解步数**，即将时间演化算子：
$$
e^{-i H t}
$$
近似分解为 **2** 个较小步骤，以提高近似精度。  

```python
@qml.qnode(dev)
def circuit():
    qml.ApproxTimeEvolution(H, t, n)
    return [qml.expval(qml.PauliZ(i)) for i in range(2)]
```

- **定义量子电路** `circuit()`：

  - `qml.ApproxTimeEvolution(H, t, n)`：
  
    - 计算哈密顿量 \( H \) 在时间 \( t \) 内的近似时间演化：
      $$
      e^{-i H t} \approx \prod_{i} e^{-i H_i t / n}
      $$

  - `return [qml.expval(qml.PauliZ(i)) for i in range(2)]`：

    - 测量量子比特 \( i \) 在 \( Z \) 方向的期望值：
      $$
      \langle Z_0 \rangle, \quad \langle Z_1 \rangle
      $$

```python
print(qml.draw(circuit, level="device")())
```
- **绘制量子电路**，以 `device` 级别的详细视图图展示。

## 量子门解析

- `RX(1.00)`：
  - **Pauli-X 旋转门**，作用于 **第 0 量子比特**：
    $$
    R_X(\theta) = e^{-i \frac{\theta}{2} X}
    $$

- `RZ(1.00)`：
  - **Pauli-Z 旋转门**，作用于 **第 1 量子比特**：
    $$
    R_Z(\theta) = e^{-i \frac{\theta}{2} Z}
    $$

- `rXX(0.50)`：
  - **Ising XX 交互门**，作用于 **第 0 和 1 号量子比特**：
    $$
    e^{-i \frac{t}{n} X_0 X_1}
    $$

  - 这表示量子比特 0 和 1 之间存在相互作用（如 Ising 模型）。



In [6]:
def circ(theta):
    qml.RX(theta, wires=0)
    qml.Hadamard(wires=1)
    qml.CNOT(wires=[0, 1])


@qml.qnode(dev)
def circuit(param):
    circ(param)
    return [qml.expval(qml.PauliZ(i)) for i in range(2)]


print(qml.draw(circuit)(0.5))

0: ──RX(0.50)─╭●─┤  <Z>
1: ──H────────╰X─┤  <Z>


- `qml.RX(theta, wires=0)`：
  - 在 **第 0 号量子比特** 上施加 **RX 旋转门**，旋转角度为 `theta`：
    $$
    R_X(\theta) = e^{-i \frac{\theta}{2} X}
    $$
  - 这是一个绕 **X 轴** 的旋转门。

- `qml.Hadamard(wires=1)`：
  - 在 **第 1 号量子比特** 上施加 **Hadamard 门**：
    $$
    H = \frac{1}{\sqrt{2}} 
    \begin{bmatrix} 
    1 & 1 \\ 
    1 & -1 
    \end{bmatrix}
    $$
  - 将量子比特 1 置于叠加态：
    $$
    |0\rangle \rightarrow \frac{|0\rangle + |1\rangle}{\sqrt{2}}, \quad 
    |1\rangle \rightarrow \frac{|0\rangle - |1\rangle}{\sqrt{2}}
    $$

- `qml.CNOT(wires=[0, 1])`：
  - 施加 **受控非门 (CNOT)**：
    - **控制比特**：量子比特 `0`
    - **目标比特**：量子比特 `1`
    - 如果控制比特 `0` 处于 $|1\rangle$ 状态，则翻转比特 `1`。  

```python
@qml.qnode(dev)
def circuit(param):
    circ(param)
    return [qml.expval(qml.PauliZ(i)) for i in range(2)]
```

- `@qml.qnode(dev)`：
  - 将 `circuit` **变为一个可执行的量子电路**，在 `dev` 设备上运行。

- `circ(param)`：
  - 调用 **子电路** `circ(theta)`，其中 `theta` 由 `param` 传递进来。

- `return [qml.expval(qml.PauliZ(i)) for i in range(2)]`：
  - 测量两个量子比特的 `Pauli-Z` 期望值：
    $$
    \langle Z_0 \rangle, \quad \langle Z_1 \rangle
    $$
  - 这用于获取最终状态在 **Z** 方向的投影结果。

  ```python
  print(qml.draw(circuit)(0.5))
  ```


`qml.draw(circuit)(0.5)` **绘制电路**，并将 `param = 0.5` 作为 `theta` 传递给 `circ(theta)`。

**最终输出的是电路结构图。**

电路中的 **量子门** 作用如下：

1. **第 0 号量子比特** (`wire 0`)

   - `RX(0.50)`：在量子比特 0 上施加 **RX 旋转门**：
     $$
     R_X(0.50) = e^{-i \frac{0.50}{2} X}
     $$

   **作用**：绕 X 轴旋转 0.50 弧度，使得量子比特从 `|0⟩` 状态进入一个 **叠加态**。

   - 🔘 **(控制 CNOT 门)**：
     - 该比特 **作为 CNOT 门的控制比特**。
     - 仅当该比特处于 `|1⟩` 状态时，目标比特 (`wire 1`) 会发生翻转。

   -  `<Z>` **(测量)**：
     - 对该比特进行 `Pauli-Z` 期望值测量：
       $$
       \langle Z_0 \rangle
       $$
   
   **作用**：获取该比特最终在 Z 方向的投影值。
### **第 1 号量子比特 (`wire 1`)**

-  **H (Hadamard 门)**：
  - 作用于 **第 1 号量子比特**：
    $$
    H = \frac{1}{\sqrt{2}} 
    \begin{bmatrix} 
    1 & 1 \\ 
    1 & -1 
    \end{bmatrix}
    $$
  - **作用**：使 `|0⟩` 和 `|1⟩` 进入等概率的叠加态：
    $$
    |0\rangle \rightarrow \frac{|0\rangle + |1\rangle}{\sqrt{2}}
    $$

-  **X (CNOT 目标比特)**：
  - 该比特是 CNOT 门的 **目标比特**。
  - 如果 **控制比特 (`wire 0`) 处于 `|1⟩`**，则该比特的状态翻转：
    - `|0⟩ → |1⟩`
    - `|1⟩ → |0⟩`
  - **作用**：在比特 0 和比特 1 之间 **创建量子纠缠**。

-  `<Z>` **(测量)**：
  - 对该比特进行 **Pauli-Z 期望值测量**：
    $$
    \langle Z_1 \rangle
    $$
  - **作用**：获取该比特最终在 **Z** 方向的投影值。


In [7]:
@qml.qnode(dev)
def circuit(params, **kwargs):
    qml.layer(circ, 3, params)
    return [qml.expval(qml.PauliZ(i)) for i in range(2)]


print(qml.draw(circuit)([0.3, 0.4, 0.5]))

0: ──RX(0.30)─╭●──RX(0.40)─╭●──RX(0.50)─╭●─┤  <Z>
1: ──H────────╰X──H────────╰X──H────────╰X─┤  <Z>


`@qml.qnode(dev)`：把 `circuit()` 变成 PennyLane 量子计算节点（QNode），可以在 `dev` 设备（量子模拟器）上运行。

`qml.layer(circ, 3, params)`：

- **核心**：让 `circ` 这个量子电路函数被重复执行 **3 次**。
- `params = [0.3, 0.4, 0.5]`，表示 **每一层的 RX 旋转门使用不同的角度**。

**返回值**：

- 计算 **两个量子比特在 `Pauli-Z` 方向的期望值**：
  $$
  \langle Z_0 \rangle, \quad \langle Z_1 \rangle
  $$
